**Шаг 1: Загрузка и предобработка данных**

В этом этапе мы:

Загружаем датасет MovieLens 1M (или 100K, если RAM ограничена).

Загружаем таблицу ratings.csv (пользователь, фильм, оценка).

Проверяем структуру данных (размер, пропущенные значения).

Преобразуем данные в нужный формат для Surprise.

In [1]:
# === Шаг 1: Загрузка данных и предобработка ===

# Импортируем библиотеки
import pandas as pd

# Загружаем данные
from google.colab import files
print("Загрузите файл ratings.csv")
uploaded = files.upload()

# Читаем датасет
ratings = pd.read_csv("ratings.csv")

# Просматриваем первые строки
print("\nПример данных из ratings.csv:")
display(ratings.head())

# Проверяем структуру
print("\nРазмер датасета:", ratings.shape)
print("\nИнформация о данных:")
print(ratings.info())

# Проверяем пропущенные значения
print("\nПропущенные значения в данных:")
print(ratings.isnull().sum())


Загрузите файл ratings.csv


Saving ratings.csv to ratings.csv

Пример данных из ratings.csv:


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931



Размер датасета: (100836, 4)

Информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB
None

Пропущенные значения в данных:
userId       0
movieId      0
rating       0
timestamp    0
dtype: int64


**Что мы видим:**

Датасет MovieLens 100K загружен успешно (100 836 записей, 4 столбца).

Включает колонки:

userId – ID пользователя.

movieId – ID фильма.

rating – Оценка фильма (от 0.5 до 5.0).

timestamp – Временная метка (не нужна для модели).

Пропущенных значений нет – отлично!

#Шаг 2: Перевод данных в формат Surprise

Сейчас мы преобразуем данные в формат библиотеки Surprise, который нужен для работы моделей.

In [3]:
# === Шаг 2: Подготовка данных для Surprise ===

!pip install scikit-surprise


from surprise import Dataset, Reader

# Определяем формат данных для Surprise
reader = Reader(rating_scale=(0.5, 5.0))  # Оценки от 0.5 до 5.0
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Проверяем количество пользователей и фильмов
num_users = ratings['userId'].nunique()
num_movies = ratings['movieId'].nunique()

print(f"\nКоличество пользователей: {num_users}")
print(f"Количество фильмов: {num_movies}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505182 sha256=aea93fe23f622e662c867f01e8d7a8801d19015ed32935bb88a5fdf17772f07e
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise

Количество пользователей: 610
Количество фильмов: 9724


#Выводы по подготовке данных

Библиотека surprise установлена, и данные успешно преобразованы

Количество пользователей: 610
Количество фильмов: 9 724
Теперь мы можем приступить к выбору модели и обучению.

#Шаг 3: Выбор модели и обучение

В библиотеке Surprise есть несколько алгоритмов коллаборативной фильтрации. Мы попробуем три разных подхода:

SVD (Singular Value Decomposition) – один из самых популярных алгоритмов.

KNNBasic (User-based KNN) – на основе схожести пользователей.

Slope One – простая, но эффективная модель.

#Цель – найти модель, которая достигнет RMSE ≤ 0.87 на 5-кратной кросс-валидации.

In [5]:
# === Шаг 3: Выбор модели и кросс-валидация ===

from surprise import SVD, KNNBasic, SlopeOne
from surprise.model_selection import cross_validate

# Список моделей для тестирования
models = {
    "SVD": SVD(),
    "KNNBasic": KNNBasic(),
    "SlopeOne": SlopeOne()
}

# Оценка моделей через 5-фолд кросс-валидацию
results = {}
for name, model in models.items():
    print(f"\nТестируем модель: {name}")
    cv_results = cross_validate(model, data, cv=5, measures=['RMSE'], verbose=True)
    results[name] = round(cv_results['test_rmse'].mean(), 4)

# Выводим результаты
print("\n📊 Итоговые RMSE для моделей:")
for model, rmse in results.items():
    print(f"{model}: {rmse}")



Тестируем модель: SVD
Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8710  0.8788  0.8651  0.8842  0.8709  0.8740  0.0067  
Fit time          1.34    1.36    4.49    2.88    1.52    2.32    1.23    
Test time         0.22    0.10    0.34    0.17    0.26    0.22    0.08    

Тестируем модель: KNNBasic
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9489  0.9458  0.9401  0.9612  0.9377  0.9467  0.0083  
Fit time          0.10    0.12 

#Выводы по обученным моделям

RMSE для моделей (на 5-кратной кросс-валидации):

SVD: 0.874 ✅ (почти достигли цели 0.87!)

SlopeOne: 0.903 ❌ (хуже, но близко)

KNNBasic: 0.9467 ❌ (хуже всех, убираем из рассмотрения)

#Теперь попробуем оптимизировать гиперпараметры SVD, чтобы снизить RMSE до 0.87 или ниже.

In [7]:
# === Шаг 4: Оптимизация модели SVD ===

from surprise.model_selection import GridSearchCV
from surprise import SVD

# Гиперпараметры для тестирования
param_grid = {
    'n_factors': [50, 100, 150],  # Количество скрытых факторов
    'reg_all': [0.02, 0.05, 0.1],  # Регуляризация
    'lr_all': [0.002, 0.005, 0.01]  # Скорость обучения
}

# Поиск лучших параметров
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5, joblib_verbose=1)
gs.fit(data)

# Лучшие параметры и результат
best_params = gs.best_params['rmse']
best_rmse = gs.best_score['rmse']

print("\n🔍 Лучшие параметры SVD:", best_params)
print(f"✅ Лучший RMSE после оптимизации: {best_rmse}")


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:  1.0min



🔍 Лучшие параметры SVD: {'n_factors': 50, 'reg_all': 0.05, 'lr_all': 0.01}
✅ Лучший RMSE после оптимизации: 0.8600502323535585


#🎯 Цель достигнута! RMSE = 0.860 ✅

✅ Лучшие параметры для модели SVD:

n_factors: 50

reg_all: 0.05

lr_all: 0.01

✅ Лучший RMSE после оптимизации: 0.860

#Финальные выводы:

Лучшей моделью оказалась SVD – после оптимизации она достигла RMSE = 0.860.

SlopeOne и KNNBasic показали худшие результаты, поэтому не были выбраны.

Подбор гиперпараметров улучшил точность модели – исходное RMSE было 0.874, а после настройки 0.860.

**Задача выполнена успешно – модель готова к использованию в рекомендациях!**